# Using FACE to get feature attributions in MNIST

## Image classification problem - MNIST dataset

### This notebook presents the utilization of FACE to get the feature attributions observed in a trained Multilayer Perceptron for a sample using heatmaps.

## Load general libraries

In [1]:
# -*- coding: utf-8 -*-

#!git clone https://github.com/CarlesBou/mlpxai.git
#!cd mlpxai/src/examples/notebooks

'''
Adjust import paths if needed
'''
import os
import sys

#print(os.getcwd())

sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.insert(0, os.path.dirname(os.getcwd()))

#sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

In [2]:
'''
Import general libraries
'''
import keras
from keras.utils.generic_utils import get_custom_objects
from keras.datasets import mnist
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
import tensorflow as tf

import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load FACE module from mlpxai library

In [3]:
from mlpxai.explainers.face.kerasmlp import get_face_contrib # Key funtion to perform FACE feature attribution computation
from mlpxai.utils.visualize import plot_MNIST_digit          # Support module to displat feature attrubution bars

## Load the dataset of a tabular classification problem

In [4]:
'''
Classfication tabular problem using 'Liver Disorder' dataset
'''
seed = 1
# np.seed = 1
# random.seed = 1
# tf.random.set_seed(1)
keras.utils.set_random_seed(seed)

ds_name = 'MNIST'

(X_train, y_train), (X_test, y_test) = mnist.load_data()

num_inputs = 28 * 28
num_outputs = 10

dataset_name = 'MNIST'

input_layer = Input(shape=(num_inputs,))
hidden_layer = Dense(num_inputs * 2, activation='relu')(input_layer)
hidden_layer = Dense(5, activation='relu')(hidden_layer)
output_layer = Dense(num_outputs, activation='linear')(hidden_layer)

model_version = 0

validation_split = 0.1 
epochs = 10

use_saved_model_weights = True # Use pretrained model if available

NameError: name 'mnist' is not defined

## Define the Keras network model

In [ ]:
'''
Define the network model structure (2 hidden layers using ReLUs)
'''
input_layer = Input(shape=(num_inputs,))
hidden_layer = Dense(num_inputs * 2, activation='relu')(input_layer)
hidden_layer = Dense(5, activation='relu')(hidden_layer)
output_layer = Dense(num_outputs, activation='linear')(hidden_layer)

model_version = 0

validation_split = 0.1 
epochs = 10

use_saved_model_weights = True # Use pretrained model if available

'''
Scale the input in range [0, 1]
'''
X_train = X_train.astype(np.float32) / 255. #Transform integer pixel values to [0,1]
X_train = X_train.reshape(-1, num_inputs)     #Transfor image matrix into vector
X_test = X_test.astype(np.float32) / 255.   #Transform integer pixel values to [0,1]
X_test = X_test.reshape(-1, num_inputs)       #Transfor image matrix into vector
    
y_train_categorical = to_categorical(y_train, num_outputs).astype(np.float32)

## Compile and train the model using the training dataset

In [5]:
'''
Define and compile the Keras model
'''
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='nadam', loss='mean_squared_error')

'''
As the Delta Elevators dataset takes some time to be compiled,
we save the model as soon it is compiled for the first time to avoid
long time waiting.
'''
weights_file_name = f'{ds_name}_classfication_seed_{seed}_epochs_{epochs}.weights.h5'

if use_saved_model_weights:
    if os.path.isfile(weights_file_name):
        model.load_weights(weights_file_name)
        print(f'Using pretrained classification model weights from file {weights_file_name}')
    else:
        print('Training classification model ... ', end='')
        my_fit = model.fit(X_train, y_train_categorical,
                           epochs=epochs, validation_split=validation_split, verbose=0)
        model.save_weights(weights_file_name)
        print('OK')
else:
    print('Training classification model ... ', end='')
    my_fit = model.fit(X_train, y_train_categorical,
                       epochs=epochs, validation_split=validation_split, verbose=0)
    print('OK')

NameError: name 'input_layer' is not defined

## Run a model prediction over the test set to show the performance of the model

In [20]:
print('Generating predictions for test data ... ', end='')

predictions = model.predict(X_test, verbose=0)

y_mlp = np.argmax(predictions, axis=1)

print('OK')

accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)

print(f'Test data accuracy = {accuracy:.5f}\n')

Generating predictions for test data ... OK
Test data accuracy = 0.8846



## Compute the feature attribution using FACE and display digits

In [26]:
'''
Select some samples to explain and display their saliency maps
'''
samples = [8535, 8330, 8841]

for sample in samples:

    print(f'Computing FACE attributions for test sample {sample} ground/net/FACE={y_test[sample]}/{y_mlp[sample]}/', end='')
    
    FACE_contrib = get_face_contrib(X_test[sample], model)
    
    y_FACE = np.argmax(np.sum(FACE_contrib, axis=1))
    
    print(f'{y_FACE} ... OK')
    
    y_truth = y_test[sample]
        
    contrib = np.array([FACE_contrib[y_truth][1:]])
    
    plot_MNIST_digit(X_test[sample], contrib, resize=1)

FACE attributions for sample 43
  Class 0: [ 0.95484032  0.32896234 -0.18163942 -0.05544087 -0.1640083  -0.11076466]
  Class 1: [-1.10983027 -0.4779491   0.00851536 -0.0255018   0.15161711  0.08601518]
